In [1]:
import pandas as pd
from pandas import DataFrame
from pandas.core.frame import DataFrame

import os

In [2]:
CARPETA = r'C:\Users\cesar\OneDrive - Universidad de Burgos\TFG\Repo\CIC_IOT-20250124T124300Z-001\CIC_IOT'

# Listar los archivos de la carpeta
df = DataFrame(os.listdir(CARPETA), columns=['filenames'])

df['prefix'] = df['filenames'].apply(lambda x: x.rsplit('_', 1)[0])
# Agrupar los archivos por tipo de ataque
df_grouped = df.groupby('prefix')['filenames'].apply(list).reset_index()

df_grouped

,prefix,filenames
0,flows_aggregated,[flows_aggregated_Recon-OSScan.csv]
1,flows_aggregated_BenignTraffic,"[flows_aggregated_BenignTraffic_0.csv, flows_a..."
2,flows_aggregated_DDoS-HTTP,[flows_aggregated_DDoS-HTTP_Flood.csv]
3,flows_aggregated_DDoS-SYN,"[flows_aggregated_DDoS-SYN_0.csv, flows_aggreg..."
4,flows_aggregated_DoS-TCP_Flood,"[flows_aggregated_DoS-TCP_Flood_0.csv, flows_a..."
5,flows_aggregated_Recon,[flows_aggregated_Recon_PortScan.csv]


In [3]:
def filter_filenames(df, text):
    return df[df['prefix'].str.contains(text)]['filenames'].iloc[0]


filtered_filenames = filter_filenames(df_grouped, 'Benign')


dfs: list[DataFrame] = [pd.read_csv(os.path.join(CARPETA, file),
                                    sep=';', header=0, index_col=False,
                                    usecols=lambda x: x.endswith('_2') and
                                    (x.startswith('num_flows') or x.startswith('packets_forward_mean')))
                        for file in filtered_filenames]
df_combined: DataFrame = pd.concat(dfs, ignore_index=True)

df_combined.mean().rename('Mean Values').to_frame().T

,num_flows_2,packets_forward_mean_2
Mean Values,25.820068,60.471138
